In [1]:
import numpy as np
from copy import deepcopy

In [19]:
path_to_data = 'Data/Stocks/a.us.txt'

with open(path_to_data, 'r') as stream:
    data_lines = stream.readlines()

print(data_lines[0])
print(data_lines[1])

X = list()

data_lines = data_lines[1:]

for line in data_lines:
    try:
        X.append(float(line.split(',')[1]))
    except:
        print(line)

X = np.array(X)

Date,Open,High,Low,Close,Volume,OpenInt

1999-11-18,30.713,33.754,27.002,29.702,66277506,0



In [40]:
class ARMA_OGD:
    """
    AutoRegressive Moving Average model with parameter estimation by Online Gradient Descent.
    """
    
    def __init__(self, k = 3, q = 3, eta = 0.001, threshold = 0.001, logger=False, plot=False):
        """
        
        """
        self.k = k
        self.q = q
        self.eta = eta
        self.threshold = threshold
        self.w_ma = np.random.randn(k)
        self.w_ar = np.random.randn(q)
        
        self.logger = logger
        self.plot = plot
        
    def fit(self, X, n_epochs = 10):
        max_kq = max(self.k, self.q)
        k_offset = max_kq - self.k
        q_offset = max_kq - self.q
        
        w_ma = self.w_ma
        w_ar = self.w_ar
        
        
        n = X.shape[0]
        
        Y = np.random.randn(n)
        
        ma_changes = list()
        ar_changes = list()
        errors = list()
        
        for i in range(n_epochs):
            old_ma = w_ma.copy()
            old_ar = w_ar.copy()
            
            for j in np.random.permutation(n - max_kq):
                curr_idx = j + max_kq
                
                x_prev = X[j+k_offset : curr_idx]
                y_prev = Y[j+q_offset : curr_idx]
                
                pred = np.dot(x_prev, w_ma) + np.dot(y_prev, w_ar)
                
                Y[curr_idx] = pred
                err = X[curr_idx] - pred
                
            ma_change = np.linalg.norm(old_ma - w_ma)
            ma_changes.append(ma_change)
            
            ar_change = np.linalg.norm(old_ar - w_ar)
            ar_changes.append(ar_change)
            
            mean_abs_error = np.mean(np.abs(Y[max_kq:] - X[max_kq:]))
            errors.append(mean_abs_error)
            
            if ar_change < self.threshold and ma_change < self.threshold:
                break
        
        self.w_ma = w_ma
        self.w_ar = w_ar
    
    def predict(self, X):
        max_kq = max(self.k, self.q)
        
        n = X.shape[0]
        Y = np.zeros(n)
        
        print(max_kq)
        print(n - max_kq)
        
        for i in np.arange(n - max_kq):
            curr_idx = i + max_kq
            
            x_prev = X[i + max_kq - self.k : curr_idx]
            y_prev = Y[i + max_kq - self.q : curr_idx]
            
            Y[curr_idx] = np.dot(x_prev, self.w_ma) + np.dot(y_prev, self.w_ar)
            
        
        return Y

In [41]:
model = ARMA_OGD()

In [42]:
model.fit(X)

In [50]:
model.predict(X[16:20])

3
1


array([  0.        ,   0.        ,   0.        ,  11.28442728])